In [ ]:
import numpy as np
import pandas as pd
import seaborn as snb
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

df = pd.read_csv("/content/drive/MyDrive/x/datasets /E commerce dataset/flipkart/flipkart1.csv")


In [ ]:
print(df.head)

<bound method NDFrame.head of        Unnamed: 0                                   _id  actual_price  \
0               0  fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a        2999.0   
1               1  893e6980-f2a0-531f-b056-34dd63fe912c        1499.0   
2               2  eb4c8eab-8206-59d0-bcd1-a724d96bf74f        2999.0   
3               3  3f3f97bb-5faf-57df-a9ff-1af24e2b1045        2999.0   
4               4  750caa3d-6264-53ca-8ce1-94118a1d8951        2999.0   
...           ...                                   ...           ...   
29995       29995  3705c6bd-6f23-529a-9b64-15b8fc568efa        2299.0   
29996       29996  f2a683e4-634d-5a11-8950-2d56b924576d         799.0   
29997       29997  1efa858c-1360-59a6-9624-bb04eddb492c        4999.0   
29998       29998  9fdfdd22-487b-599b-8be6-5dd00eb987c5        3125.0   
29999       29999  a5396fab-753c-5cd7-bad9-050964f95a0f        3125.0   

       average_rating brand                  category            crawled_at  \
0             

In [ ]:
print(df.describe)

<bound method NDFrame.describe of        Unnamed: 0                                   _id  actual_price  \
0               0  fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a        2999.0   
1               1  893e6980-f2a0-531f-b056-34dd63fe912c        1499.0   
2               2  eb4c8eab-8206-59d0-bcd1-a724d96bf74f        2999.0   
3               3  3f3f97bb-5faf-57df-a9ff-1af24e2b1045        2999.0   
4               4  750caa3d-6264-53ca-8ce1-94118a1d8951        2999.0   
...           ...                                   ...           ...   
29995       29995  3705c6bd-6f23-529a-9b64-15b8fc568efa        2299.0   
29996       29996  f2a683e4-634d-5a11-8950-2d56b924576d         799.0   
29997       29997  1efa858c-1360-59a6-9624-bb04eddb492c        4999.0   
29998       29998  9fdfdd22-487b-599b-8be6-5dd00eb987c5        3125.0   
29999       29999  a5396fab-753c-5cd7-bad9-050964f95a0f        3125.0   

       average_rating brand                  category            crawled_at  \
0         

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       30000 non-null  int64  
 1   _id              30000 non-null  object 
 2   actual_price     29137 non-null  float64
 3   average_rating   27554 non-null  float64
 4   brand            27932 non-null  object 
 5   category         30000 non-null  object 
 6   crawled_at       30000 non-null  object 
 7   description      29059 non-null  object 
 8   discount         18017 non-null  object 
 9   images           30000 non-null  object 
 10  out_of_stock     30000 non-null  bool   
 11  pid              30000 non-null  object 
 12  product_details  30000 non-null  object 
 13  seller           28259 non-null  object 
 14  selling_price    29998 non-null  float64
 15  sub_category     30000 non-null  object 
 16  title            30000 non-null  object 
 17  url         

In [ ]:
df['average_rating'].fillna(df['average_rating'].mean(), inplace=True)

target_variable = 'average_rating'

# List of columns to be used as features
feature_columns = ['brand', 'category', 'description', 'discount', 'actual_price', 'out_of_stock', 'pid', 'product_details', 'seller', 'selling_price', 'sub_category', 'title', 'url']


features = df[feature_columns]
target = df[target_variable]

features_train,features_test,target_train, target_test=train_test_split(features,target, test_size=0.3,random_state=42)

print("training set shape",features_train.shape,target_train.shape)
print("testing set shape:",features_test.shape,target_test.shape)

training set shape (21000, 13) (21000,)
testing set shape: (9000, 13) (9000,)


In [ ]:
numerical_columns = features.select_dtypes(include=['float64']).columns
categorical_columns = features.select_dtypes(include=['object']).columns

# Create transformers for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Use mean imputation for numerical columns
    ('num', OneHotEncoder(handle_unknown='ignore'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Use most frequent imputation for categorical columns
    ('cat', OneHotEncoder(handle_unknown='ignore'))
])

# Create a ColumnTransformer to apply transformers to specific columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],
    remainder='passthrough'
)

# Apply preprocessing to the training and testing sets
features_train_preprocessed = preprocessor.fit_transform(features_train)
features_test_preprocessed = preprocessor.transform(features_test)

print("Preprocessed Training Set Shape:", features_train_preprocessed.shape)
print("Preprocessed Testing Set Shape:", features_test_preprocessed.shape)


Preprocessed Training Set Shape: (21000, 70437)
Preprocessed Testing Set Shape: (9000, 70437)


In [ ]:
model = KNeighborsRegressor(n_neighbors=5)

# Train the model
model.fit(features_train_preprocessed, target_train)

# Make predictions on the test set
predictions = model.predict(features_test_preprocessed)

# Evaluate mean squared error
mae = mean_absolute_error(target_test, predictions)
print("Mean Absolute Error:", mae)

threshold = 3.5

# Convert ratings to binary labels
predictions_binary = (predictions > threshold).astype(int)
target_test_binary = (target_test > threshold).astype(int)

# Use accuracy_score and confusion_matrix for binary classification
accuracy = accuracy_score(target_test_binary, predictions_binary)
print("Accuracy:", accuracy)

conf_matrix = confusion_matrix(target_test_binary, predictions_binary)
print("Confusion Matrix:")
print(conf_matrix)

Mean Absolute Error: 0.2131415315380707
Accuracy: 0.8814444444444445
Confusion Matrix:
[[2330  573]
 [ 494 5603]]


In [ ]:
features['content'] = features['brand'] + ' ' + features['category'] + ' ' + features['discount'] + ' ' + \
                       features['seller'] + ' ' + features['sub_category'] + ' ' + features['title']
features['content'] = features['content'].fillna('')
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(features['content'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

<ipython-input-28-4ed9e6d9083e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['content'] = features['brand'] + ' ' + features['category'] + ' ' + features['discount'] + ' ' + \
<ipython-input-28-4ed9e6d9083e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['content'] = features['content'].fillna('')


In [ ]:
def get_recommendations(item_id):
    idx = df[df['pid'] == item_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    item_indices = [i[0] for i in sim_scores]
    recommended_items = df.iloc[item_indices]
    return recommended_items


In [ ]:
# def get_recommendations_by_inputs(df, rating=None, brand=None, category=None, discount=None, seller=None, selling_price=None, subcategory=None):
#     # Convert relevant columns to float type
#     numeric_columns = ['average_rating', 'selling_price']
#     df[numeric_columns] = df[numeric_columns].astype(float)

#     # Filter the DataFrame based on input values
#     filtered_df = df.copy()
#     if rating is not None:
#         filtered_df = filtered_df[filtered_df['average_rating'] >= float(rating)]
#     if brand is not None:
#         filtered_df = filtered_df[filtered_df['brand'] == brand]
#     if category is not None:
#         filtered_df = filtered_df[filtered_df['category'] == category]
#     if discount is not None:
#         filtered_df = filtered_df[filtered_df['discount'] == discount]
#     if seller is not None:
#         filtered_df = filtered_df[filtered_df['seller'] == seller]
#     if selling_price is not None:
#         filtered_df = filtered_df[filtered_df['selling_price'] <= float(selling_price)]
#     if subcategory is not None:
#         filtered_df = filtered_df[filtered_df['sub_category'] == subcategory]

#     # Check if the filtered DataFrame is empty
#     if filtered_df.empty:
#         return "No products match the specified criteria."

#     # Reset the index of the filtered DataFrame
#     filtered_df = filtered_df.reset_index(drop=True)

#     # Define numerical columns
#     numerical_columns = ['average_rating', 'selling_price']

#     # Filter out non-numeric columns
#     filtered_columns = numerical_columns + ['brand', 'category', 'discount', 'seller', 'sub_category', 'title']

#     # Remove columns that are not in the DataFrame
#     filtered_columns = [col for col in filtered_columns if col in filtered_df.columns]

#     # Combine transformers using ColumnTransformer
#     preprocessor = ColumnTransformer(
#         transformers=[
#             ('num', numerical_transformer, numerical_columns),
#             ('cat', categorical_transformer, filtered_columns),
#         ]
#     )

#     # Fit and transform the data with the pipeline
#     X_filtered = preprocessor.fit_transform(filtered_df[filtered_columns])

#     # Compute the cosine similarity matrix for the sample
#     cosine_sim_sample = cosine_similarity(X_filtered, X_filtered)

#     # Generate recommendations based on the cosine similarity matrix
#     idx = 0  # Using index 0 assuming filtered_df is not empty
#     sim_scores = list(enumerate(cosine_sim_sample[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:11]
#     item_indices = [i[0] for i in sim_scores]
#     recommendations = filtered_df.iloc[item_indices]

#     return recommendations


In [ ]:
def get_recommendations_by_inputs(rating=None, brand=None, category=None, discount=None, seller=None, selling_price=None, subcategory=None):
    global df
    # Convert relevant columns to float type
    numeric_columns = ['average_rating', 'selling_price']
    df[numeric_columns] = df[numeric_columns].astype(float)

    # Filter the DataFrame based on input values
    filtered_df = df.copy()
    if rating is not None:
        filtered_df = filtered_df[filtered_df['average_rating'] >= float(rating)]
    if brand is not None:
        filtered_df = filtered_df[filtered_df['brand'] == brand]
    if category is not None:
        filtered_df = filtered_df[filtered_df['category'] == category]
    if discount is not None:
        filtered_df = filtered_df[filtered_df['discount'] == discount]
    if seller is not None:
        filtered_df = filtered_df[filtered_df['seller'] == seller]
    if selling_price is not None:
        filtered_df = filtered_df[filtered_df['selling_price'] <= float(selling_price)]
    if subcategory is not None:
        filtered_df = filtered_df[filtered_df['sub_category'] == subcategory]

    # Check if the filtered DataFrame is empty
    if filtered_df.empty:
        return pd.DataFrame()  # Return an empty DataFrame

    # Reset the index of the filtered DataFrame
    filtered_df = filtered_df.reset_index(drop=True)

    # Define numerical columns
    numerical_columns = ['average_rating', 'selling_price']

    # Filter out non-numeric columns
    filtered_columns = numerical_columns + ['brand', 'category', 'discount', 'seller', 'sub_category', 'title']

    # Remove columns that are not in the DataFrame
    filtered_columns = [col for col in filtered_columns if col in filtered_df.columns]

    # Combine transformers using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_columns),
            ('cat', categorical_transformer, filtered_columns),
        ]
    )

    # Fit and transform the data with the pipeline
    X_filtered = preprocessor.fit_transform(filtered_df[filtered_columns])

    # Compute the cosine similarity matrix for the sample
    cosine_sim_sample = cosine_similarity(X_filtered, X_filtered)

    # Generate recommendations based on the cosine similarity matrix
    idx = 0  # Using index 0 assuming filtered_df is not empty
    sim_scores = list(enumerate(cosine_sim_sample[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    item_indices = [i[0] for i in sim_scores]
    recommendations = filtered_df.iloc[item_indices]



    return recommendations


In [ ]:
# # Example: Get recommendations for an item with pid 'example_pid'
# example_pid = 'TKPFCZ9EHFCY5Z4Y'  # Replace with a valid item ID
# recommendations = get_recommendations(example_pid)
# print(recommendations)

In [ ]:
# # Example: Get recommendations based on rating and brand
# recommendations = get_recommendations_by_inputs(df ,subcategory='Bottomwear',brand='adidas Origina',selling_price=2159)
# print(recommendations)


In [ ]:
import joblib

joblib.dump(model, 'ecommerce.joblib')


['ecommerce.joblib']

In [ ]:
def get_recommendations(item_id):
    # Check if the item_id exists in the dataframe
    if df[df['pid'] == item_id].empty:
        return pd.DataFrame()  # Return an empty DataFrame

    # Get the index of the item_id
    idx = df[df['pid'] == item_id].index[0]

    # Get item-based recommendations based on the index
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    item_indices = [i[0] for i in sim_scores]
    recommended_items = df.iloc[item_indices]

    return recommended_items



In [ ]:
# Example: Get recommendations for an item with pid 'example_pid'
example_pid = 'TKPFCZ9EHFCY5Z4Y'  # Replace with a valid item ID
recommendations = get_recommendations(example_pid)
print(recommendations)

    Unnamed: 0                                   _id  actual_price  \
2            2  eb4c8eab-8206-59d0-bcd1-a724d96bf74f        2999.0   
3            3  3f3f97bb-5faf-57df-a9ff-1af24e2b1045        2999.0   
5            5  9af4ac75-230b-5d51-8bbf-2ffb19aaa763        4499.0   
19          19  2cc3eb4c-42b5-5bc5-9eb6-0f438f45dd58        2999.0   
27          27  a4a976d7-483d-5fff-8e75-cdc481138215        4599.0   
1            1  893e6980-f2a0-531f-b056-34dd63fe912c        1499.0   
8            8  ab4f0cca-6a84-51e1-b7e8-523724db8728        1499.0   
9            9  f8a91e89-c5db-5fa5-9428-1f580a72d05e        1499.0   
23          23  4bd87d93-7851-5536-9804-9c6d502809dc        4599.0   
7            7  a33cf2e2-c5e4-5f06-b3aa-8f46a6fd1aef        1499.0   

    average_rating brand                  category            crawled_at  \
2              3.9  York  Clothing and Accessories  02/10/2021, 20:11:52   
3              3.9  York  Clothing and Accessories  02/10/2021, 20:11:53   
5